In [1]:
# import 
from amumo import model as am_model
from amumo import data as am_data
from amumo import widgets as am_widgets

C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\uma

In [2]:
# Data Helpers
def get_data_helper(dataset, filters=[], method=any):
    all_images, all_prompts = dataset.get_filtered_data(filters, method=method)
    print(len(all_images))

    dataset_name = dataset.name
    if len(filters) > 0:
        dataset_name = dataset_name + '_filter-' + method.__name__ + '_' + '-'.join(filters)
    else:
        dataset_name = dataset_name + '_size-%i'%len(all_images)

    return all_images, all_prompts, dataset_name

# # load dataset
# dataset = am_data.DiffusionDB_Dataset(path="2m_first_1k", batch_size=100) # data helper for the diffusionDB dataset; for the interactive prototype, we only use a random subset of 100 samples
# all_images, all_prompts = dataset.get_data()
# cache_name = 'diffusiondb_random_100' # path used to cache the results

# Load Data
data_path = '../../../../../Data/'
# subset of mscoco validation data
dataset_mscoco_val = am_data.MSCOCO_Val_Dataset(path=data_path+'mscoco/validation', batch_size=100) 
mscoco_val_images, mscoco_val_prompts, mscoco_val_dataset_name = get_data_helper(dataset_mscoco_val, filters=[], method=any)
mscoco_val_dataset_name


loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
100


'MSCOCO-Val_size-100'

In [3]:
import os
import torch
import numpy as np
import json
from amumo.CLOOB_local.clip import clip as cloob
from amumo.CLOOB_local.clip.model import CLIPGeneral
from amumo.CLOOB_local.cloob_training import model_pt, pretrained

class Custom_CLOOB_Model(am_model.CLIPModelInterface):
    available_models = ['clip_infoLOOB', 'clip', 'cloob_infoNCE', 'cloob']
    model_name = ''

    def __init__(self, name='clip_infoLOOB', device='cpu') -> None:
        super().__init__(name, device)

        self.model_name += name

        ckpt_name = name + '_rn50_cc_epoch_31.pt' # cloob_infoNCE_rn50_cc_epoch_31.pt
        checkpoint_path = 'checkpoints/cloob_tests/' + ckpt_name
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model_config_file = os.path.join('../amumo/CLOOB_local/training/model_configs/', checkpoint['model_config_file'])

        print('Loading model from', model_config_file)
        assert os.path.exists(model_config_file), 'config file does not exist'
        with open(model_config_file, 'r') as f:
            model_info = json.load(f)
        self.model = CLIPGeneral(**model_info)
        self.model.eval()

        sd = checkpoint["state_dict"]
        sd = {k[len('module.'):]: v for k, v in sd.items()}
        if 'logit_scale_hopfield' in sd:
            sd.pop('logit_scale_hopfield', None)
        self.model.load_state_dict(sd)

        self.preprocess = cloob._transform(self.model.visual.input_resolution, is_train=False)

        self.logit_scale = self.model.logit_inv_tau


    def encode_image(self, images):
        images = [self.preprocess(i) for i in images]
        image_input = torch.tensor(np.stack(images)).to(self.device)
        return self.model.encode_image(image_input).float().cpu()

    def encode_text(self, texts):
        text_tokens = cloob.tokenize(texts).to(self.device)
        return self.model.encode_text(text_tokens).float().cpu()

In [4]:
am_widgets.CLIPExplorerWidget(mscoco_val_dataset_name, mscoco_val_images, mscoco_val_prompts, models=[Custom_CLOOB_Model(name='clip'), Custom_CLOOB_Model(name='clip_infoLOOB'), Custom_CLOOB_Model(name='cloob'), Custom_CLOOB_Model(name='cloob_infoNCE')])

Loading model from ../amumo/CLOOB_local/training/model_configs/RN50.json
Loading model from ../amumo/CLOOB_local/training/model_configs/RN50.json
Loading model from ../amumo/CLOOB_local/training/model_configs/RN50.json
Loading model from ../amumo/CLOOB_local/training/model_configs/RN50.json
found cached embeddings for MSCOCO-Val_size-100_clip_clip


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('clip', 'c…

found cached embeddings for MSCOCO-Val_size-100_clip_infoLOOB_clip_infoLOOB
found cached embeddings for MSCOCO-Val_size-100_cloob_cloob
found cached embeddings for MSCOCO-Val_size-100_cloob_infoNCE_cloob_infoNCE
found cached embeddings for MSCOCO-Val_size-100_clip_clip
found cached embeddings for MSCOCO-Val_size-100_clip_infoLOOB_clip_infoLOOB
found cached embeddings for MSCOCO-Val_size-100_cloob_infoNCE_cloob_infoNCE
found cached embeddings for MSCOCO-Val_size-100_cloob_cloob
found cached embeddings for MSCOCO-Val_size-100_clip_clip
found cached embeddings for MSCOCO-Val_size-100_clip_infoLOOB_clip_infoLOOB
found cached embeddings for MSCOCO-Val_size-100_cloob_infoNCE_cloob_infoNCE
found cached embeddings for MSCOCO-Val_size-100_cloob_cloob
found cached embeddings for MSCOCO-Val_size-100_cloob_infoNCE_cloob_infoNCE
 True
found cached embeddings for MSCOCO-Val_size-100_cloob_cloob
found cached embeddings for MSCOCO-Val_size-100_clip_infoLOOB_clip_infoLOOB
found cached embeddings for MS

In [5]:
# ['clip_infoLOOB', 'clip', 'cloob_infoNCE', 'cloob']
am_widgets.CLIPComparerWidget(mscoco_val_dataset_name, mscoco_val_images, mscoco_val_prompts, models=[Custom_CLOOB_Model(name='clip'), Custom_CLOOB_Model(name='clip_infoLOOB'), Custom_CLOOB_Model(name='cloob'), Custom_CLOOB_Model(name='cloob_infoNCE')], zmin=-0.4, zmax=1) 

Loading model from ../amumo/CLOOB_local/training/model_configs/RN50.json
Loading model from ../amumo/CLOOB_local/training/model_configs/RN50.json
Loading model from ../amumo/CLOOB_local/training/model_configs/RN50.json
Loading model from ../amumo/CLOOB_local/training/model_configs/RN50.json
True [False, False, False, False] {'clip': <__main__.Custom_CLOOB_Model object at 0x0000024C28D55F40>, 'clip_infoLOOB': <__main__.Custom_CLOOB_Model object at 0x0000024C324A1F70>, 'cloob': <__main__.Custom_CLOOB_Model object at 0x0000024C32E1AF70>, 'cloob_infoNCE': <__main__.Custom_CLOOB_Model object at 0x0000024C324C8F70>}
found cached embeddings for MSCOCO-Val_size-100_clip_clip


c:\users\christina\repositories\icg\researchstay\amumo\amumo\widgets.py:844: FutureWarning:

The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.

c:\users\christina\repositories\icg\researchstay\amumo\amumo\widgets.py:844: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



found cached embeddings for MSCOCO-Val_size-100_clip_infoLOOB_clip_infoLOOB
found cached embeddings for MSCOCO-Val_size-100_cloob_cloob
found cached embeddings for MSCOCO-Val_size-100_cloob_infoNCE_cloob_infoNCE


CLIPComparerWidget(children=(HoverWidget(children=(VBox(children=(HTML(value='', layout=Layout(width='300px'))…